In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

22/02/27 20:46:37 WARN Utils: Your hostname, Vinoths-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.11 instead (on interface en0)
22/02/27 20:46:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/27 20:46:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark.version

'3.2.1'

In [5]:
!wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-01.csv

--2022-02-27 20:48:34--  https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-01.csv
Resolving nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)... 52.217.164.113
Connecting to nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)|52.217.164.113|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 752335705 (717M) [text/csv]
Saving to: ‘fhvhv_tripdata_2021-01.csv’

fhvhv_tripdata_2021 100%[===================>] 717.48M  20.0MB/s    in 39s     

2022-02-27 20:49:13 (18.5 MB/s) - ‘fhvhv_tripdata_2021-01.csv’ saved [752335705/752335705]



In [6]:
!wc -l fhvhv_tripdata_2021-01.csv

 11908469 fhvhv_tripdata_2021-01.csv


In [7]:
df = spark.read \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2021-01.csv')

In [8]:
df.schema

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,StringType,true),StructField(dropoff_datetime,StringType,true),StructField(PULocationID,StringType,true),StructField(DOLocationID,StringType,true),StructField(SR_Flag,StringType,true)))

In [9]:
!head -n 1001 fhvhv_tripdata_2021-01.csv > head.csv

In [10]:
import pandas as pd

In [11]:
df_pandas = pd.read_csv('head.csv')

In [12]:
df_pandas.dtypes

hvfhs_license_num        object
dispatching_base_num     object
pickup_datetime          object
dropoff_datetime         object
PULocationID              int64
DOLocationID              int64
SR_Flag                 float64
dtype: object

In [13]:
spark.createDataFrame(df_pandas).schema

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,StringType,true),StructField(dropoff_datetime,StringType,true),StructField(PULocationID,LongType,true),StructField(DOLocationID,LongType,true),StructField(SR_Flag,DoubleType,true)))

In [14]:
from pyspark.sql import types

In [15]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [16]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-01.csv')

In [17]:
df = df.repartition(24)

In [18]:
df.write.parquet('fhvhv/2021/01/')

22/02/27 20:53:03 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/02/27 20:53:05 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/02/27 20:53:06 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [19]:
df = spark.read.parquet('fhvhv/2021/01/')

In [20]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



In [21]:
from pyspark.sql import functions as F

In [22]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02880|2021-01-02 20:47:47|2021-01-02 21:13:43|         250|         258|   null|
|           HV0003|              B02835|2021-01-04 19:40:17|2021-01-04 19:57:29|         112|          68|   null|
|           HV0003|              B02764|2021-01-03 21:33:00|2021-01-03 22:01:58|         225|         170|   null|
|           HV0003|              B02871|2021-01-02 20:20:00|2021-01-02 20:28:10|         149|         165|   null|
|           HV0003|              B02872|2021-01-01 03:17:59|2021-01-01 03:48:44|          74|          39|   null|
|           HV0005|              B02510|2021-01-03 14:58:04|2021-01-03 15:33:02|

In [23]:
def crazy_stuff(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    elif num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'

In [24]:
crazy_stuff('B02884')

's/b44'

In [25]:
crazy_stuff_udf = F.udf(crazy_stuff, returnType=types.StringType())

In [26]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .withColumn('base_id', crazy_stuff_udf(df.dispatching_base_num)) \
    .select('base_id', 'pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
    .show()

+-------+-----------+------------+------------+------------+
|base_id|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-------+-----------+------------+------------+------------+
|  a/b40| 2021-01-02|  2021-01-02|         250|         258|
|  s/b13| 2021-01-04|  2021-01-04|         112|          68|
|  e/acc| 2021-01-03|  2021-01-03|         225|         170|
|  a/b37| 2021-01-02|  2021-01-02|         149|         165|
|  e/b38| 2021-01-01|  2021-01-01|          74|          39|
|  e/9ce| 2021-01-03|  2021-01-03|         138|         181|
|  s/b44| 2021-01-03|  2021-01-03|         213|          60|
|  e/b3e| 2021-01-05|  2021-01-05|          79|         138|
|  e/acc| 2021-01-01|  2021-01-01|          69|         168|
|  a/b37| 2021-01-02|  2021-01-02|         136|         119|
|  e/a39| 2021-01-05|  2021-01-05|          33|         138|
|  e/b38| 2021-01-01|  2021-01-01|          82|         135|
|  a/b37| 2021-01-05|  2021-01-05|          91|          61|
|  e/b3c| 2021-01-02|  2

In [27]:
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID') \
  .filter(df.hvfhs_license_num == 'HV0003')

DataFrame[pickup_datetime: timestamp, dropoff_datetime: timestamp, PULocationID: int, DOLocationID: int]